In [1]:
import sys
sys.path.append("..")

import os

In [2]:
import taipy as tp
import taipy.gui.builder as tgb
from taipy.gui import Gui
from source.helpers.types import StrPath
from source.gui.helpers import *
from source.steps.analysis.summary_pipe import Summary_Runner
from source.gui.configuration.config import *

In [3]:
selections = {}
local = True

summary_path_scheduled_in_quantification = "."
summary_selection_tree_scheduled_in_quantification = []
summary_select_folder_scheduled_in_quantification = False

summary_path_scheduled_in_annotation = "."
summary_selection_tree_scheduled_in_annotation = []
summary_select_folder_scheduled_in_annotation = False


summary_params = Summary_Runner()
summary_params.scheduled_in = {}
summary_params_scheduled_in = {"quantification": None, "annotation": None}

print(summary_params_scheduled_in)

scenario = tp.create_scenario(ms_analysis_config, name="Default")


def print_v_in(*args):
	print(summary_selection_tree_scheduled_in_annotation)
	print("\n")
	print(selections)
	print("\n")
	print(summary_params_scheduled_in)
	print("\n")
	print(summary_params.scheduled_in)



def create_file_selection(process: str, param_attribute_in: str = "scheduled_in", execution_key_in: str = None, out_node: str = ""):
	selections.update({f"{process}_{param_attribute_in}_{execution_key_in}": []})

	def update_dict(state, name, value):
		dictionary = get_attribute_recursive(state, f"{process}_params.{param_attribute_in}")
		dictionary.update({execution_key_in: value})
		set_attribute_recursive(state, f"{process}_params.{param_attribute_in}", dictionary, refresh=True)

	def construct_selection_tree(state, path: StrPath = None, tree_id: str = f"{process}_{param_attribute_in}_{execution_key_in}"):
		path = path if path else get_attribute_recursive(state, f"{process}_path_{param_attribute_in}_{execution_key_in}")

		if path != ".":
			selections[tree_id] = add_path_to_tree(selections[tree_id], path)

			pruned_tree = path_nester.prune_lca(nested_paths=selections[tree_id])
			set_attribute_recursive(state, f"{process}_selection_tree_{param_attribute_in}_{execution_key_in}", pruned_tree)

	with tgb.layout(columns="1 2 2", columns__mobile="1", gap="5%"):
		# In
		with tgb.part():
			with tgb.part(render="{local}"):
				tgb.button(
					"Select in",
					on_action=lambda state: construct_selection_tree(
						state,
						open_file_folder(
							select_folder=get_attribute_recursive(
								state, f"{process}_select_folder_{param_attribute_in}_{execution_key_in}"
							)
						),
					),
				)
			with tgb.part(render="{not local}"):
				tgb.file_selector(
					f"{{{process}_path_{param_attribute_in}_{execution_key_in}}}",
					label="Select in",
					extensions="*",
					drop_message=f"Drop files/folders for {process} here:",
					multiple=True,
					on_action=lambda state: construct_selection_tree(state),
				)
			tgb.toggle(f"{{{process}_select_folder_{param_attribute_in}_{execution_key_in}}}", label="Select folder")
		
		# Update dictionary nested
		if execution_key_in:
			# Should be f"{{{process}_params.{param_attribute_in}.{execution_key_in}}}", currently not possible in taipy
			param_variable_in = f"{{{process}_params_{param_attribute_in}.{execution_key_in}}}" 
			update_method = lambda state, name, value: update_dict(state, name, value)
		else:
			param_variable_in = f"{{{process}_params.{param_attribute_in}}}"
			update_method = None

		tgb.tree(
			param_variable_in,
			lov=f"{{{process}_selection_tree_{param_attribute_in}_{execution_key_in}}}",
			label=f"Select in for {process}",
			filter=True,
			multiple=True,
			expanded=True,
			on_change=update_method
		)
		tgb.button(
			"X", on_action=print_v_in
		)

		# Out
		with tgb.part():
			with tgb.part():
				with tgb.part(render="{local}"):
					tgb.button(
						"Select out",
						on_action=lambda state: set_attribute_recursive(
							state,
							f"{process}_params.scheduled_out",
							open_file_folder(select_folder=True),
							refresh=True,
						),
					)
				with tgb.part(render="{not local}"):
					tgb.file_download(
						"{None}",
						active=f"{{scenario.data_nodes['{out_node}'].is_ready_for_reading}}",
						label="Download results",
						on_action=lambda state, id, payload: download_data_node_files(
							state, out_node
						),
					)

{'quantification': None, 'annotation': None}
[2025-01-13 13:08:16.205][Taipy][INFO] Updating configuration with command-line arguments...
[2025-01-13 13:08:16.207][Taipy][INFO] Managing application's version...
[2025-01-13 13:08:16.209][Taipy][INFO] Development mode: Clean all entities of version 40baa7a0-2684-47cd-a0cf-2288ddbf25c4
[2025-01-13 13:08:16.414][Taipy][INFO] Checking application's version...
[2025-01-13 13:08:16.415][Taipy][INFO] Blocking configuration update...


In [ ]:
with tgb.Page() as root:
    create_file_selection("summary", execution_key_in="quantification")
    create_file_selection("summary", execution_key_in="annotation")

gui = Gui(page=root)

if __name__ == "__main__":
    gui.run(port=4999, run_browser=False)

[2025-01-13 13:08:22.572][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2025-01-13 13:08:23.402][Taipy][INFO]  * Server starting on http://127.0.0.1:4999


[]


{'summary_scheduled_in_quantification': [{'id': '1', 'label': 'mnt', 'children': [{'id': '2', 'label': 'd', 'children': [{'id': '3', 'label': 'mine2sirius_pipe', 'children': [{'id': '4', 'label': 'source', 'children': [{'id': '5', 'label': '/mnt/d/mine2sirius_pipe/source/dumpsterfire', 'children': []}]}]}]}]}], 'summary_scheduled_in_annotation': []}


{'quantification': [{'id': '5', 'label': '/mnt/d/mine2sirius_pipe/source/dumpsterfire', 'children': []}], 'annotation': None}


{'quantification': [{'id': '5', 'label': '/mnt/d/mine2sirius_pipe/source/dumpsterfire', 'children': []}]}


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter.ttk import Frame, Label, Button, Checkbutton

class InstallerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Installer")
        self.root.geometry("400x300")
        
        # Initialize variables
        self.current_page = 0
        self.install_dir = tk.StringVar(value="C:/Program Files/MyApp")  # Default installation directory
        self.accept_var = tk.BooleanVar()
        self.component_vars = {
            "Component A": tk.BooleanVar(value=True),
            "Component B": tk.BooleanVar(value=False),
            "Component C": tk.BooleanVar(value=False),
        }
        
        # List of pages
        self.pages = [
            self.create_license_page,
            self.create_component_page,
            self.create_installation_location_page,
        ]
        
        # Main frame for dynamic content
        self.main_frame = Frame(root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Navigation buttons
        self.nav_frame = Frame(root)
        self.nav_frame.pack(fill=tk.X, pady=10)
        
        self.prev_button = Button(self.nav_frame, text="Previous", command=self.previous_page, state=tk.DISABLED)
        self.prev_button.pack(side=tk.LEFT, padx=5)
        
        self.next_button = Button(self.nav_frame, text="Next", command=self.next_page)
        self.next_button.pack(side=tk.RIGHT, padx=5)
        
        # Load the first page
        self.load_page()
    
    def load_page(self):
        """Clears the main frame and loads the current page."""
        for widget in self.main_frame.winfo_children():
            widget.destroy()
        self.pages[self.current_page]()
        
        # Update navigation button states
        self.prev_button.config(state=tk.NORMAL if self.current_page > 0 else tk.DISABLED)
        self.next_button.config(
            text="Install" if self.current_page == len(self.pages) - 1 else "Next"
        )
    
    def next_page(self):
        """Handles navigation to the next page."""
        if self.current_page == 0 and not self.accept_var.get():
            messagebox.showwarning("License Agreement", "You must accept the license agreement to proceed.")
            return
        if self.current_page < len(self.pages) - 1:
            self.current_page += 1
            self.load_page()
        else:
            self.install()
    
    def previous_page(self):
        """Handles navigation to the previous page."""
        if self.current_page > 0:
            self.current_page -= 1
            self.load_page()
    
    def create_license_page(self):
        """Creates the license agreement page."""
        Label(self.main_frame, text="License Agreement", font=("Arial", 14, "bold")).pack(pady=10)
        
        license_text = tk.Text(self.main_frame, height=10, wrap=tk.WORD)
        license_text.insert(tk.END, "Please read the license agreement carefully.\n\n[License terms here...]")
        license_text.config(state=tk.DISABLED)
        license_text.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        Checkbutton(self.main_frame, text="I accept the license agreement", variable=self.accept_var).pack(pady=5)
    
    def create_component_page(self):
        """Creates the component selection page."""
        Label(self.main_frame, text="Select Components to Install", font=("Arial", 14, "bold")).pack(pady=10)
        
        for component, var in self.component_vars.items():
            Checkbutton(self.main_frame, text=component, variable=var).pack(anchor="w", padx=20)
    
    def create_installation_location_page(self):
        """Creates the installation location selection page."""
        Label(self.main_frame, text="Select Installation Location", font=("Arial", 14, "bold")).pack(pady=10)
        
        # Display current installation directory
        entry = tk.Entry(self.main_frame, textvariable=self.install_dir, width=40)
        entry.pack(pady=5, padx=10)
        
        # Button to browse for a directory
        browse_button = Button(self.main_frame, text="Browse", command=self.browse_directory)
        browse_button.pack(pady=5)
    
    def browse_directory(self):
        """Opens a file dialog to select the installation directory."""
        directory = filedialog.askdirectory(initialdir=self.install_dir.get(), title="Select Installation Directory")
        if directory:
            self.install_dir.set(directory)
    
    def install(self):
        """Final installation process."""
        selected_components = [name for name, var in self.component_vars.items() if var.get()]
        messagebox.showinfo(
            "Installation",
            f"Installing components: {', '.join(selected_components)}\n"
            f"Installation directory: {self.install_dir.get()}",
        )
        self.root.quit()

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = InstallerApp(root)
    root.mainloop()
